## テクスチャ特徴量抽出

In [1]:
import os, sys
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from texture_extractor import TextureExtractor

import logging
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)

# データシートにて，データパスが書かれているカラムを設定
PATH_COLUMNS = "path"

### データシートの読み込み

In [2]:
dataseet = pd.read_csv("./sc_data_seet.csv", index_col=PATH_COLUMNS)
dataseet.head()

,f,k
path,,
images/gs_2700_5500.tiff,0.02700,0.055
images/gs_2352_5500.tiff,0.02352,0.055
images/gs_2865_5500.tiff,0.02865,0.055
images/gs_2016_5500.tiff,0.02016,0.055
images/gs_2511_5500.tiff,0.02511,0.055


### テクスチャ抽出インスタンス作成

In [3]:
setting = {
    "binWidth" : 25,
}
texture_extractor = TextureExtractor(setting=setting)

### テクスチャ抽出の実行

In [12]:
df = pd.DataFrame()
for index, row in dataseet.iterrows():
    image = np.array( Image.open(index).convert('L') )
    image_array = np.array(image)
    mask_array = np.ones(image_array.shape)
    result = texture_extractor.get_texture_feature(image_array, mask_array)
    df = pd.concat([df, pd.DataFrame(result, index=[index])])
    
print(df.shape)
df.head()

(30, 93)


,FO-10Percentile,FO-90Percentile,FO-Energy,FO-Entropy,FO-InterquartileRange,FO-Kurtosis,FO-Maximum,FO-MeanAbsoluteDeviation,FO-Mean,FO-Median,...,GLDM-GrayLevelNonUniformity,GLDM-GrayLevelVariance,GLDM-HighGrayLevelEmphasis,GLDM-LargeDependenceEmphasis,GLDM-LargeDependenceHighGrayLevelEmphasis,GLDM-LargeDependenceLowGrayLevelEmphasis,GLDM-LowGrayLevelEmphasis,GLDM-SmallDependenceEmphasis,GLDM-SmallDependenceHighGrayLevelEmphasis,GLDM-SmallDependenceLowGrayLevelEmphasis
images/gs_2700_5500.tiff,80.0,168.0,1.052519e+09,3.405113,67.0,1.522951,185.0,30.242184,122.210449,119.0,...,6576.874054,11.438502,44.526978,14.701660,752.526672,3.733832,0.162998,0.209504,8.639307,0.015143
images/gs_2352_5500.tiff,82.0,184.0,1.302279e+09,3.794278,70.0,1.757117,222.0,33.882882,135.608383,139.0,...,5066.031006,14.891563,97.885620,15.466980,1769.269104,0.593822,0.029833,0.227243,18.966320,0.004950
images/gs_2865_5500.tiff,84.0,162.0,9.269245e+08,3.267057,51.0,2.015939,189.0,25.867374,115.155777,106.0,...,8035.591309,8.945549,34.515472,18.980835,481.739380,3.860386,0.126095,0.162945,6.490233,0.009743
images/gs_2016_5500.tiff,85.0,210.0,1.729606e+09,4.078885,71.0,2.160657,243.0,38.617567,155.876160,165.0,...,4253.227844,21.024605,193.645294,16.637695,4167.176147,0.180204,0.011377,0.215087,31.781469,0.002400
images/gs_2511_5500.tiff,80.0,174.0,1.149166e+09,3.513627,72.0,1.526665,202.0,32.400909,127.447327,128.0,...,6303.044678,13.051097,99.326157,15.231079,1771.556778,0.342950,0.020424,0.194658,17.758160,0.003248


### パッチの特徴量の中央値を取得

In [13]:
texture_extractor = TextureExtractor(clip_size=64, num_sample=50)

In [21]:
df = pd.DataFrame()
for index, row in dataseet.iterrows():
    image = np.array( Image.open(index).convert('L') )
    image_array = np.array(image)
    result = texture_extractor.get_median_texture_feature_from_patch(image_array)
    df = pd.concat([df, pd.DataFrame(result, index=[index])])

print(df.shape)
df.head()

(30, 93)


,FO-10Percentile,FO-90Percentile,FO-Energy,FO-Entropy,FO-InterquartileRange,FO-Kurtosis,FO-Maximum,FO-MeanAbsoluteDeviation,FO-Mean,FO-Median,...,GLDM-GrayLevelNonUniformity,GLDM-GrayLevelVariance,GLDM-HighGrayLevelEmphasis,GLDM-LargeDependenceEmphasis,GLDM-LargeDependenceHighGrayLevelEmphasis,GLDM-LargeDependenceLowGrayLevelEmphasis,GLDM-LowGrayLevelEmphasis,GLDM-SmallDependenceEmphasis,GLDM-SmallDependenceHighGrayLevelEmphasis,GLDM-SmallDependenceLowGrayLevelEmphasis
images/gs_2700_5500.tiff,80.0,168.0,65768938.5,3.404218,67.0,1.527695,182.0,30.278509,122.188110,119.0,...,411.718262,11.425302,44.540894,14.559570,741.930298,3.582121,0.161740,0.215958,8.924885,0.015770
images/gs_2352_5500.tiff,82.0,183.0,79975180.0,3.674380,72.0,1.690541,202.5,34.109095,134.249146,137.5,...,341.223145,14.652255,87.506348,15.232422,1528.564209,0.847933,0.040664,0.234072,17.382410,0.006282
images/gs_2865_5500.tiff,84.0,162.0,57630264.5,3.235227,49.0,2.131790,186.0,25.480702,114.922241,105.0,...,523.198730,8.702844,34.135742,18.987549,452.347046,3.673538,0.117053,0.166099,6.732334,0.009783
images/gs_2016_5500.tiff,85.0,205.5,105303410.0,4.026638,72.0,2.098877,229.5,38.324788,154.107544,164.0,...,272.601807,20.788011,148.415039,15.635254,3055.127075,0.462343,0.025917,0.224839,25.973057,0.004393
images/gs_2511_5500.tiff,80.0,174.0,71848685.0,3.445178,71.0,1.500968,189.0,32.064883,127.573975,128.0,...,409.566895,12.693936,62.051270,15.111328,1090.994995,1.278417,0.063220,0.197934,10.672011,0.009789


### テクスチャ特徴量と特性をマージしたデータフレーム作成

In [15]:
df = pd.merge(dataseet, df, left_index=True, right_index=True)

In [17]:
df.head()

,f,k,FO-10Percentile,FO-90Percentile,FO-Energy,FO-Entropy,FO-InterquartileRange,FO-Kurtosis,FO-Maximum,FO-MeanAbsoluteDeviation,...,GLDM-GrayLevelNonUniformity,GLDM-GrayLevelVariance,GLDM-HighGrayLevelEmphasis,GLDM-LargeDependenceEmphasis,GLDM-LargeDependenceHighGrayLevelEmphasis,GLDM-LargeDependenceLowGrayLevelEmphasis,GLDM-LowGrayLevelEmphasis,GLDM-SmallDependenceEmphasis,GLDM-SmallDependenceHighGrayLevelEmphasis,GLDM-SmallDependenceLowGrayLevelEmphasis
images/gs_2700_5500.tiff,0.02700,0.055,80.0,168.0,65702944.0,3.402735,67.0,1.525951,182.0,30.257086,...,411.365723,11.430972,44.471558,14.435303,748.659790,3.611574,0.162432,0.210769,8.710825,0.015590
images/gs_2352_5500.tiff,0.02352,0.055,82.0,183.0,79755425.0,3.716334,70.5,1.705813,204.5,34.234462,...,333.121338,14.831551,87.022827,15.095947,1554.476196,0.793590,0.039393,0.227239,17.400874,0.006233
images/gs_2865_5500.tiff,0.02865,0.055,84.0,162.0,57451071.5,3.215714,49.0,2.117386,186.0,25.411663,...,528.907715,8.675424,33.830688,19.164795,456.471680,3.646795,0.118120,0.165134,6.568105,0.009742
images/gs_2016_5500.tiff,0.02016,0.055,86.5,205.0,105237787.0,4.004682,68.5,2.163799,229.5,37.591751,...,282.270508,19.809954,147.439575,15.941895,3138.769409,0.443468,0.025780,0.219360,24.040120,0.004561
images/gs_2511_5500.tiff,0.02511,0.055,80.0,174.0,71655663.5,3.441141,71.0,1.496841,189.5,32.206178,...,408.314209,12.866426,64.442871,14.815186,1156.587036,1.152621,0.059227,0.202450,10.647700,0.008465


In [18]:
texture_extractor.feature_class_func.keys()

dict_keys(['FO', 'GLCM', 'GLSZM', 'GLRLM', 'NGTDM', 'GLDM'])

In [19]:
target_key = "GLCM"
target_columns = list(filter(lambda item: target_key in item, df.columns))
df[target_columns].head()

,GLCM-Autocorrelation,GLCM-ClusterProminence,GLCM-ClusterShade,GLCM-ClusterTendency,GLCM-Contrast,GLCM-Correlation,GLCM-DifferenceAverage,GLCM-DifferenceEntropy,GLCM-DifferenceVariance,GLCM-Id,...,GLCM-Imc2,GLCM-InverseVariance,GLCM-JointAverage,GLCM-JointEnergy,GLCM-JointEntropy,GLCM-MCC,GLCM-MaximumProbability,GLCM-SumAverage,GLCM-SumEntropy,GLCM-SumSquares
images/gs_2700_5500.tiff,43.409315,2969.986049,31.857865,43.690060,2.002453,0.912556,1.080168,1.766008,0.775448,0.593102,...,0.962437,0.463964,5.742605,0.029363,5.476603,0.913015,0.070264,11.485210,4.319311,11.426761
images/gs_2352_5500.tiff,86.025054,5331.654205,-44.215092,57.038918,2.195800,0.924722,1.108738,1.749105,0.793280,0.592675,...,0.976669,0.456817,8.636492,0.022304,5.869723,0.926367,0.059313,17.272984,4.652312,14.822556
images/gs_2865_5500.tiff,33.159490,2348.916412,122.884425,33.246182,1.462591,0.915200,0.886846,1.622183,0.645663,0.643643,...,0.956987,0.468253,5.017080,0.047615,5.153203,0.915935,0.152555,10.034160,4.134438,8.673222
images/gs_2016_5500.tiff,145.829262,13359.308831,-302.927877,76.276143,2.347016,0.941030,1.113700,1.929088,1.052615,0.599771,...,0.979495,0.448348,11.259158,0.017888,6.312427,0.945599,0.054198,22.518316,4.955593,19.640522
images/gs_2511_5500.tiff,63.316576,3713.316145,-3.298054,49.270470,2.211050,0.913838,1.131473,1.701817,0.720200,0.588472,...,0.977619,0.430949,7.166447,0.035313,5.350807,0.914575,0.098231,14.332894,4.317215,12.867228


### 特徴量テーブルのダウンロード

In [20]:
df.to_csv("feature_table.csv")